In [1]:
! pip install -U langchain-nomic langchain_community tiktoken langchainhub langchain langgraph tavily-python llama-index llama-parse llama-index-llms-groq llama-index-embeddings-fastembed fastembed  llama-index-vector-stores-qdrant

  Using cached fastembed-0.3.0-py3-none-any.whl (48 kB)


In [2]:
# pdf data
!wget "https://lawcommission.gov.np/en/wp-content/uploads/2021/01/Constitution-of-Nepal.pdf"
!wget "https://www.lawcommission.gov.np/en/wp-content/uploads/2021/01/Nepal-Citizenship-Act-2063-2006.pdf"

--2024-06-16 16:22:49--  https://lawcommission.gov.np/en/wp-content/uploads/2021/01/Constitution-of-Nepal.pdf
Resolving lawcommission.gov.np (lawcommission.gov.np)... 202.45.146.189
Connecting to lawcommission.gov.np (lawcommission.gov.np)|202.45.146.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3516131 (3.4M) [application/pdf]
Saving to: ‘Constitution-of-Nepal.pdf’

Constitution-of-Nep 100%[===================>]   3.35M  1.69MB/s    in 2.0s    

2024-06-16 16:22:52 (1.69 MB/s) - ‘Constitution-of-Nepal.pdf’ saved [3516131/3516131]

--2024-06-16 16:22:53--  https://www.lawcommission.gov.np/en/wp-content/uploads/2021/01/Nepal-Citizenship-Act-2063-2006.pdf
Resolving www.lawcommission.gov.np (www.lawcommission.gov.np)... 202.45.146.189
Connecting to www.lawcommission.gov.np (www.lawcommission.gov.np)|202.45.146.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 453830 (443K) [application/pdf]
Saving to: ‘Nepal-Citizenship-Act-2063-

In [7]:
import os
llama_parse_api_key=os.environ["LLAMA_PARSE_API_KEY"]

llx-pnnTKw8sNbmLqRTHd8CX9Jvn9OlPll7vVzwXfGfHiNOPkxhM


In [8]:
import os
import nest_asyncio
nest_asyncio.apply()
llama_parse_api_key=os.environ["LLAMA_PARSE_API_KEY"]

from llama_parse import LlamaParse
llama_parse_documents = LlamaParse(api_key=llama_parse_api_key, result_type="markdown").load_data(["/content/Constitution-of-Nepal.pdf","/content/Nepal-Citizenship-Act-2063-2006.pdf"])

Parsing files: 100%|██████████| 2/2 [00:03<00:00,  1.97s/it]


In [9]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

In [18]:
# model
from llama_index.llms.groq import Groq

os.environ["GROQ_API_KEY"]='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'
llm1 = Groq(model="Llama3-70b-8192", api_key="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")

In [19]:
from llama_index.core import Settings
Settings.llm=llm1
Settings.embed_model=embed_model

In [22]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
import qdrant_client

client = qdrant_client.QdrantClient(
    "https://157df9e2-c80f-418f-bfb5-d75133f04154.us-east4-0.gcp.cloud.qdrant.io",
    api_key="8J1gEvOSEqii300JBP5pugw5E35kEbELwsleEJbMjuXS9rfICC6E5A",
)

vector_store = QdrantVectorStore(client=client, collection_name="legal_documents")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index =VectorStoreIndex.from_documents(documents=llama_parse_documents,storage_context=storage_context,show_progress=True)

Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/91 [00:00<?, ?it/s]

In [15]:
from IPython.display import display, Markdown

In [36]:
query_engine = index.as_query_engine()
response = query_engine.query("can a foreign citizen get nepali citizenship by staying 5 years or longer in nepal?")
display(Markdown(response.response))

According to the provided context, a foreign citizen can acquire Nepalese citizenship by naturalization under Section 4 of the law. To be eligible, the person must have resided in Nepal for a minimum period of 15 years, among other conditions. There is no provision for acquiring Nepalese citizenship by staying 5 years or longer in Nepal.

In [37]:
retriever = index.as_retriever(search_kwargs={"k":5})

In [38]:
os.environ['LANGCHAIN_TRACING-V2']='true'
os.environ['LANGCHAIN_ENDPOINT']='https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY']='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'

In [39]:
%pip install -qU langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.0 MB/s eta 0:00:00


In [40]:
import time
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser

In [67]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0,
    model="Llama3-70b-8192",
    api_key="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"
)

In [68]:
# generation
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks.Only
    Use the following pieces of retrieved context to answer the question.If context doesnot provide information to answer that question, just say user to do web search.
    If you find it,try to explain the answer keeping the answer concise. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
start = time.time()
rag_chain = prompt | llm | StrOutputParser()
question = "how a non member of house of representative become a minister?"
docs = retriever.retrieve(question)
docs = [d.text for d in docs]
generation = rag_chain.invoke({"context":docs,"question": question})
print(generation)
end = time.time()
print(f"The time required to generate response by Router Chain in seconds:{end - start}")

The provided context does not explicitly explain how a non-member of the House of Representatives can become a minister. It appears that the context primarily focuses on the rules and procedures related to the House of Representatives, National Assembly, and various commissions. 

To answer your question, I would recommend conducting a web search or consulting relevant government documents or laws to find the specific information on how a non-member of the House of Representatives can become a minister.
The time required to generate response by Router Chain in seconds:0.9444422721862793


In [69]:
# answer grader
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether
    a generation of a llm answers the given question or not. Give a  score 'yes' or 'no' to indicate whether the question is
    fully answered or not. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
    \n ------- \n
    {generation}
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],
)
start = time.time()
answer_grader = prompt | llm | JsonOutputParser()
answer_grader_response = answer_grader.invoke({"question": question,"generation": generation})
end = time.time()
print(f"The time required to generate response by the answer grader in seconds:{end - start}")
print(answer_grader_response)

The time required to generate response by the answer grader in seconds:0.2233138084411621
{'score': 'no'}


In [72]:
# web search
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ['TAVILY_API_KEY'] = "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"
web_search_tool = TavilySearchResults(k=3)

In [73]:
from typing_extensions import TypedDict
from typing import List

### State

class GraphState(TypedDict):
    question : str
    generation : str
    web_search : str
    documents : List[str]

In [74]:
from langchain.schema import Document

In [75]:
def generate(state):

  question = state["question"]
  documents = state["documents"]

  query_engine = index.as_query_engine()
  # generation
  generation = query_engine.query(question)
  generation = generation.response
  print("GENERATING FROM CONTEXT")
  # using answer grader to grade our answer
  score = answer_grader.invoke({"question": question,"generation": generation})
  print(score['score'])
  if score['score']=='yes':
    print(" CONTEXT RESPONSE IS OK")
    return {"documents": documents, "question": question,"generation":generation}

  else:
    #state["web_search"]=='yes'
    print("DOING WEB SEARCH")
    while score['score']=='no':

      question=state["question"]
      documents=state["documents"]
      # web searching
      docs = web_search_tool.invoke({"query": question})
      print(docs)
      web_results = "\n".join([d["content"] for d in docs])
      web_results = Document(page_content=web_results)
      if documents is not None:
          documents.append(web_results)
      else:
          documents = [web_results]


      # generation
      generation = rag_chain.invoke({"context": documents, "question": question})
      print("GENERATING FROM WEB_SEARCH")
      # grading answer
      score = answer_grader.invoke({"question": question,"generation": generation})
      if score['score']=='yes':
        print("WEBSEARCH RESULT IS OK")
        return {"documents": documents, "question": question,"generation":generation}
        break


In [76]:
from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)

#workflow.add_node("websearch",web_search)
workflow.add_node("generate",generate)

In [77]:
workflow.set_entry_point("generate")
#workflow.add_edge("generate","websearch")
workflow.set_finish_point("generate")

In [78]:
app = workflow.compile()

In [83]:
from pprint import pprint
inputs = {"question":"how a person who is not a member of house of representative become minister in Nepal?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
# pprint(Markdown(value["generation"]))
display(Markdown(value["generation"]))

GENERATING FROM CONTEXT
no
DOING WEB SEARCH
[{'url': 'https://www.nbcnews.com/politics/congress/can-outsider-be-speaker-house-n441926', 'content': 'Does the speaker have to be a member of the House? The Constitution is silent on that question.'}, {'url': 'https://constitutioncenter.org/blog/the-constitution-and-the-cabinet-nomination-process', 'content': "He was the only former Senate member rejected for a cabinet position by the Senate in its history. Dick Cheney was later approved in Tower's place. Another 12 cabinet nominations have been withdrawn during the process in the Senate or didn't proceed due to Senate inaction. Most recently, the nomination of Tom Daschle by President Barack ..."}, {'url': 'https://www.idea.int/sites/default/files/publications/should-ministers-be-members-of-the-legislature.pdf', 'content': 'Having ministers in the legislature makes a more competent legislature. When ministers are members of the legislature, there will generally be a number of legislators—b

According to the context, in Nepal, a person who is not a member of the House of Representatives can become a minister if they become a Member of Parliament within 6 months of their appointment.